# teste json astronomia

In [1]:
!pip install chatterbot chatterbot_corpus spacy==3.7.4 -q
!python -m spacy download pt_core_news_sm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.5/240.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.2/920.2 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13

In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import json
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from chatterbot.languages import POR

--------------------

In [2]:
nome_arquivo_json = 'perguntas_astronomia_langchain.json'
conversa_para_treino = []

try:
    with open(nome_arquivo_json, 'r', encoding='utf-8') as f:
        dados_json = json.load(f)

    for item in dados_json:
        if 'pergunta' in item and 'resposta' in item:
            conversa_para_treino.append(item['pergunta'])
            conversa_para_treino.append(item['resposta'])
        else:
            print(f"Aviso: Item inválido no JSON (sem 'pergunta' ou 'resposta'): {item}")

except FileNotFoundError:
    print(f"Erro: Arquivo '{nome_arquivo_json}' não encontrado.")
    print("Certifique-se de que o arquivo JSON está no mesmo diretório que o script.")
    exit()
except json.JSONDecodeError:
    print(f"Erro: O arquivo '{nome_arquivo_json}' não contém um JSON válido.")
    exit()
except Exception as e:
    print(f"Ocorreu um erro inesperado ao ler o JSON: {e}")
    exit()

if not conversa_para_treino:
    print("Erro: Nenhum dado válido encontrado no JSON para treinamento.")
    exit()

In [3]:
chatbot_astro = ChatBot(
    "BotAstronomico",
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    database_uri='sqlite:///db_astronomia.sqlite3',
    logic_adapters=[
        {
            'import_path': 'chatterbot.logic.BestMatch',
            'default_response': 'Desculpe, não entendi bem. Poderia reformular?',
            'maximum_similarity_threshold': 0.90
        }
    ],
    tagger_language=POR
)

trainer_astro = ListTrainer(chatbot_astro)
print(f"Iniciando treinamento com {len(conversa_para_treino) // 2} pares de pergunta/resposta...")
trainer_astro.train(conversa_para_treino)
print("Treinamento concluído!")

Iniciando treinamento com 221 pares de pergunta/resposta...
List Trainer: [####################] 100%
Treinamento concluído!


In [4]:
print("\nOlá! Sou o Bot Astronômico. Pergunte-me algo sobre astronomia (digite 'parar' para sair).")
while True:
    try:
        mensagem = input("Você: ")
        if mensagem.lower() == "parar":
            print("Bot: Até a próxima!")
            break
        resposta = chatbot_astro.get_response(mensagem, read_only=True)
        print(f"Bot: {resposta}")
    except (KeyboardInterrupt, EOFError):
        print("\nBot: Encerrando...")
        break
    except Exception as e:
        print(f"Ocorreu um erro inesperado durante a conversa: {e}")
        break




Olá! Sou o Bot Astronômico. Pergunte-me algo sobre astronomia (digite 'parar' para sair).
Você: programa apollo
Bot: A missão Apollo 11, lançada pela NASA em julho de 1969, foi um marco histórico monumental para a humanidade. Seu objetivo principal e maior feito foi realizar o primeiro pouso tripulado na Lua. Em 20 de julho de 1969, os astronautas Neil Armstrong e Buzz Aldrin se tornaram os primeiros seres humanos a caminhar na superfície lunar, enquanto Michael Collins orbitava a Lua no Módulo de Comando. Além do feito tecnológico e exploratório sem precedentes, a missão representou o ápice da 'Corrida Espacial' entre os Estados Unidos e a União Soviética, demonstrou avanços incríveis em engenharia e ciência, e inspirou gerações em todo o mundo, unindo as pessoas em um momento de admiração pela capacidade humana.
Você: Lua Terra
Bot: As estações do ano (primavera, verão, outono, inverno) são causadas principalmente pela inclinação do eixo de rotação da Terra em relação ao seu plano o

In [ ]:
# Para limpar o banco de dados após a execução (se necessário para testes)
# chatbot_astro.storage.drop()
# print("Banco de dados do chatbot limpo.")